In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
import numpy as np
import cv2

C:\Users\82104\anaconda3\envs\py38-torch113\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### Conv layer와 FC layer의 weight 수 비교

In [2]:
fc = nn.Linear(in_features=3 * 32 * 32, out_features=4)
cnn = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)

fc_num_weight = np.prod(fc.weight.size()) + fc.bias.size()[0]
cnn_num_weight = np.prod(cnn.weight.size()) + cnn.bias.size()[0]

print("fc layer weight :", fc_num_weight)
print("cnn layer weight :", cnn_num_weight)

fc layer weight : 12292
cnn layer weight : 1792


## block 쌓기

In [3]:
class ConvBlock(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, padding):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size,
                              stride=stride, padding=padding)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, inputs):
        x = self.conv(inputs)
        x = self.relu(x)
        x = self.pool(x)
        return x

In [4]:
conv = ConvBlock(3, 64, 3, 1, "same")
temp = torch.randn(1, 3, 32, 32)
conv(temp).size()

torch.Size([1, 64, 16, 16])

## 8.4 컨볼루션 신경망 구현

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mnist_train = datasets.MNIST(root = './data/02/',
                            train=True,
                            download=True,
                            transform=transforms.ToTensor())
mnist_test = datasets.MNIST(root = './data/02/',
                            train=False,
                            download=True,
                            transform=transforms.ToTensor())
mnist_train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                           batch_size=128, shuffle=True)
mnist_test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                           batch_size=128, shuffle=False)

class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = ConvBlock(1, 6, 5, 1, "same")
        self.conv2 = ConvBlock(6, 16, 5, 1, "valid")
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, padding="valid")
        self.fc1 = nn.Linear(in_features=120, out_features=84)
        self.relu1 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(84, 10)
    
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.flatten(x, 1)
        x = self.relu1(self.fc1(x))
        out = self.fc2(x)
        return out

    
def train_step(model, loader, optimzer, criterion, device):
    model.train()
    epoch_loss = 0
    for i, (data, target) in enumerate(loader):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        if (i + 1) % 100 == 0:
            print("Train Step : {}\tLoss : {:3f}".format(i + 1, loss.item()))
    return epoch_loss / len(loader)


def evaluate_step(model, loader, criterion, device):
    model.eval()
    epoch_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            loss = criterion(output, target)
            epoch_loss += loss.item()
            correct += output.argmax(1).eq(target).sum()
    return epoch_loss / len(loader), correct / len(loader.dataset)
    
    
model = LeNet().to(device)    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(30):
    train_loss = train_step(model, mnist_train_loader, optimizer, criterion, device)
    test_loss, test_acc = evaluate_step(model, mnist_test_loader, criterion, device)
    print(f"EPOCH {epoch + 1:>3d} Train Loss {train_loss:>.3f} Test Loss {test_loss:>.3f} accuracy {test_acc:>3f}")
    print("-"*50, "\n")

Train Step : 100	Loss : 0.366707
Train Step : 200	Loss : 0.194562
Train Step : 300	Loss : 0.181697
Train Step : 400	Loss : 0.125041
EPOCH   1 Train Loss 0.359 Test Loss 0.123 accuracy 0.960900
-------------------------------------------------- 

Train Step : 100	Loss : 0.129643
Train Step : 200	Loss : 0.136415
Train Step : 300	Loss : 0.144595
Train Step : 400	Loss : 0.031688
EPOCH   2 Train Loss 0.091 Test Loss 0.071 accuracy 0.977100
-------------------------------------------------- 

Train Step : 100	Loss : 0.010697
Train Step : 200	Loss : 0.046751
Train Step : 300	Loss : 0.030403
Train Step : 400	Loss : 0.179922
EPOCH   3 Train Loss 0.066 Test Loss 0.062 accuracy 0.981000
-------------------------------------------------- 

Train Step : 100	Loss : 0.026777
Train Step : 200	Loss : 0.052842
Train Step : 300	Loss : 0.015050
Train Step : 400	Loss : 0.087437
EPOCH   4 Train Loss 0.053 Test Loss 0.050 accuracy 0.982500
-------------------------------------------------- 

Train Step : 100

In [14]:
transform = transforms.Compose(
    [transforms.ToTensor()])

cifar10_train = datasets.CIFAR10(root='./data/03', train=True,
                              download=True, transform=transform)
cifar10_test = datasets.CIFAR10(root='./data/03', train=False,
                             download=True, transform=transform)

cifar10_train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=128, shuffle=True)
cifar10_test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=128, shuffle=False)


class Cifar10_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv2 = self.build_layer(32, 32, 3, 1, "valid", 2, 0.25)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv4 = self.build_layer(64, 64, 3, 1, "valid", 2, 0.25)
        
        self.fc1 = nn.Linear(in_features=64 * 5 * 5, out_features=512)
        self.dr1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features=512, out_features=10)
        
    def build_layer(self, in_channel, out_channel, kernel_size, stride, padding, pool_size, dropout):
        layer = nn.Sequential(
            nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size, stride=stride,
                     padding=padding),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(pool_size),
            nn.Dropout(dropout)
        )
        return layer
    
    def forward(self, inputs):
        x = F.relu(self.conv1(inputs))
        x = self.conv2(x)
        x = F.relu(self.conv3(x))
        x = self.conv4(x)
        x = torch.flatten(x, 1)
        x = self.dr1(self.fc1(x))
        out = self.fc2(x)
        return out
    

model = Cifar10_CNN().to(device)    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


for epoch in range(50):
    train_loss = train_step(model, cifar10_train_loader, optimizer, criterion, device)
    test_loss, test_acc = evaluate_step(model, cifar10_test_loader, criterion, device)
    print(f"EPOCH {epoch + 1:>3d} Train Loss {train_loss:>.3f} Test Loss {test_loss:>.3f} accuracy {test_acc:>3f}")
    print("-"*50, "\n")

Files already downloaded and verified
Files already downloaded and verified
Train Step : 100	Loss : 1.704792
Train Step : 200	Loss : 1.569467
Train Step : 300	Loss : 1.513282
EPOCH   1 Train Loss 1.666 Test Loss 1.435 accuracy 0.476600
-------------------------------------------------- 

Train Step : 100	Loss : 1.361169
Train Step : 200	Loss : 1.489731
Train Step : 300	Loss : 1.224488
EPOCH   2 Train Loss 1.339 Test Loss 1.189 accuracy 0.582500
-------------------------------------------------- 

Train Step : 100	Loss : 1.274997
Train Step : 200	Loss : 1.346450
Train Step : 300	Loss : 1.158079
EPOCH   3 Train Loss 1.186 Test Loss 1.042 accuracy 0.633000
-------------------------------------------------- 

Train Step : 100	Loss : 1.177303
Train Step : 200	Loss : 1.028837
Train Step : 300	Loss : 1.128883
EPOCH   4 Train Loss 1.074 Test Loss 1.018 accuracy 0.636100
-------------------------------------------------- 

Train Step : 100	Loss : 0.943723
Train Step : 200	Loss : 0.928613
Train 

## 8.5 우편번 인식기 v.2

In [16]:
# model train
class PostNumber(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding="valid")
        self.conv2 = self.build_block(32, 32, 3, 1, "valid", 2, 0.25)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding="valid")
        self.conv4 = self.build_block(64, 64, 3, 1, "valid", 2, 0.25)
        self.fc1 = nn.Linear(in_features=1024, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=10)
        self.dr1 = nn.Dropout(0.5)
        
    def build_block(self, in_channel, out_channel, kernel_size, stride, padding, pool_size, dropout):
        block = nn.Sequential(
            nn.Conv2d(in_channels=in_channel, out_channels=out_channel, kernel_size=kernel_size,
                      stride=stride, padding=padding),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(pool_size),
            nn.Dropout(dropout)
        )
        return block
    
    def forward(self, inputs):
        x = F.relu(self.conv1(inputs))
        x = self.conv2(x)
        x = F.relu(self.conv3(x))
        x = self.conv4(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        out = self.fc2(x)
        return out
    
    
model = PostNumber().to(device)    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


for epoch in range(100):
    train_loss = train_step(model, mnist_train_loader, optimizer, criterion, device)
    test_loss, test_acc = evaluate_step(model, mnist_test_loader, criterion, device)
    print(f"EPOCH {epoch + 1:>3d} Train Loss {train_loss:>.3f} Test Loss {test_loss:>.3f} accuracy {test_acc:>3f}")
    print("-"*50, "\n")

torch.save(model.state_dict(), "./saved_model/cnn_v2.pt")

Train Step : 100	Loss : 0.156316
Train Step : 200	Loss : 0.104376
Train Step : 300	Loss : 0.203410
Train Step : 400	Loss : 0.147835
EPOCH   1 Train Loss 0.228 Test Loss 0.046 accuracy 0.984700
-------------------------------------------------- 

Train Step : 100	Loss : 0.018856
Train Step : 200	Loss : 0.058392
Train Step : 300	Loss : 0.044601
Train Step : 400	Loss : 0.033426
EPOCH   2 Train Loss 0.059 Test Loss 0.035 accuracy 0.987200
-------------------------------------------------- 

Train Step : 100	Loss : 0.068204
Train Step : 200	Loss : 0.015382
Train Step : 300	Loss : 0.039180
Train Step : 400	Loss : 0.041266
EPOCH   3 Train Loss 0.044 Test Loss 0.028 accuracy 0.991200
-------------------------------------------------- 

Train Step : 100	Loss : 0.018560
Train Step : 200	Loss : 0.021465
Train Step : 300	Loss : 0.016716
Train Step : 400	Loss : 0.038749
EPOCH   4 Train Loss 0.034 Test Loss 0.029 accuracy 0.990000
-------------------------------------------------- 

Train Step : 100

In [ ]:
model = PostNumber()
model = model.load_state_dict(torch.load("./saved_model/cnn_v2.pt"))
model = model.to(device)
model.eval()

def reset():
    global img
    img = np.ones((200, 520, 3), dtype=np.uint8) * 255
    for i in range(5):
        cv2.rectangle(img, (10 + i * 100, 50), (10 + (i + 1) * 100, 150), (0, 0, 255))
    cv2.putText(img, "e:erase s:show r:recognition q:quit", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1)
    
def grab_numerals():
    numerals = []
    for i in range(5):
        roi = img[51:149, 11+i*100:9+(i+1)*100, 0]
        roi = 255 - cv2.resize(roi, (28, 28), interpolation=cv2.INTER_CUBIC)
        numerals.append(roi)
    numerals = np.array(numerals)
    return numerals

def show():
    numerals = grab_numerals()
    plt.figure(figsize=(25, 5))
    for i in range(5):
        plt.subplot(1, 5, i + 1)
        plt.imshow(numerals[i], cmap="gray")
        plt.axis("off")
    plt.show()
    
def recognition():
    numerals = grab_numerals()
    numerals = torch.from_numpy(numerals).to(device).float() / 255.0
    numerals = numerals.permute(0, 3, 1, 2)
    res = model(numerals)
    class_id = np.argmax(res, axis=1)
    for i in range(5):
        cv2.putText(img, str(class_id[i]), (50+i*100, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)
    winsound.Beep(1000, 500)
    
brush_size = 4
l_color = (0, 0, 0)

def writing(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img, (x,y), brush_size, l_color, -1)
    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        cv2.circle(img, (x,y), brush_size, l_color, -1)
        
reset()
cv2.namedWindow("Writing")
cv2.setMouseCallback("Writing", writing)

while True:
    cv2.imshow("Writing", img)
    key = cv2.waitKey(1)
    if key == ord('e'):
        reset()
    elif key == ord('s'):
        show()
    elif key == ord('r'):
        recognition()
    elif key == ord('q'):
        break
        
cv2.destroyAllWindows()

### 8.7.2 사전 학습 모델로 자연 영상 인식

In [ ]:
weights = models.ResNet50_Weights.IMAGENET1K_V2
model = models.resnet50(weights=weights)
model.to(device)
model.eval()
preprocessor = weights.transforms()

# BGR??
img = cv2.imread("./imgs/rabbit.jpg")
x = torhc.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device).float()
x = preprocessor(x)

scores, preds_idx = torch.topk(model(x), k=5)

for i in range(5):
    class_id = preds_idx.squeeze()[i]
    cv2.putText(img, weights.meta["categories"][class_id], (10, 20+i*20)cv2.FONT_HERSHEY_SIMPLEX, 0.5,
               (255, 255, 255), 1)
cv2.imshow("Recognition result", img)
cv2.waitKeyy()
cv2.destroyAllWindows()